In [32]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import sys 
import pickle

In [34]:
movies = pd.read_csv('movie.csv')
tags = pd.read_csv('tag.csv')
ratings = pd.read_csv('rating1.csv')

In [35]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [36]:
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [37]:
ratings.head()

,userId,movieId,rating,timestamp
0,1.0,2.0,3.5,02-04-2005 23:53
1,1.0,29.0,3.5,02-04-2005 23:31
2,1.0,32.0,3.5,02-04-2005 23:33
3,1.0,47.0,3.5,02-04-2005 23:32
4,1.0,50.0,3.5,02-04-2005 23:29


In [38]:
movies['genres'] = movies['genres'].str.replace('|',' ')

In [39]:
len(movies.movieId.unique())

27278

In [40]:
len(ratings.movieId.unique())

13609

In [41]:
ratings_f = ratings.groupby('userId').filter(lambda x : len(x) >= 55)
movie_list_rating = ratings_f.movieId.unique().tolist()

In [42]:
len(ratings_f.movieId.unique())/len(movies.movieId.unique()) *100

49.53442334482

In [43]:
len(ratings_f.userId.unique())/len(ratings.userId.unique()) *100

58.80814496543994

In [44]:
movies = movies[movies.movieId.isin(movie_list_rating)]

In [45]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
mapping_file = dict(zip(movies.title.tolist(), movies.movieId.tolist()))

In [47]:
tags.drop(['timestamp'],1,inplace= True)
ratings_f.drop(['timestamp'],1,inplace= True)

In [48]:
mixed = pd.merge(movies , tags , on='movieId',how ='left')
mixed.head(3)

,movieId,title,genres,userId,tag
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1644.0,Watched
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1741.0,computer animation
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1741.0,Disney animated feature


In [49]:
mixed.fillna("", inplace = True)
mixed = pd.DataFrame(mixed.groupby('movieId')['tag'].apply( lambda x: "%s" % ' '.join(x)))
final = pd.merge(movies , mixed , on ='movieId', how ='left')
final['metadata'] = final[['tag','genres']].apply(lambda x : ' '.join(x), axis =1 )
final[['movieId','title','metadata']].head(3)

,movieId,title,metadata
0,1,Toy Story (1995),Watched computer animation Disney animated fea...
1,2,Jumanji (1995),time travel adapted from:book board game child...
2,3,Grumpier Old Men (1995),old people that is actually funny sequel fever...


In [50]:
final.shape

(13512, 5)

In [51]:
final.loc[0,"metadata"]

"Watched computer animation Disney animated feature Pixar animation TÃ©a Leoni does not star in this movie Pixar animation family Tom Hanks Pixar witty Pixar adventure animated animation clever comedy computer animation family fantasy Tom Hanks bright DARING RESCUES fanciful HEROIC MISSION humorous light rousing TOYS COME TO LIFE UNLIKELY FRIENDSHIPS warm witty animation humorous Pixar time travel Pixar Pixar animation animation kids movie Pixar Pixar Pixar witty Disney Tim Allen time travel action figure action figures Buzz Lightyear CG animation toy toys Woody animation Pixar animation Disney villian hurts toys pixar animation disney fantasy Pixar animation pixar children é˜®ä¸€é¸£ animation computer animation funny humorous Pixar Tom Hanks witty 3D Disney funny Pixar time travel Pixar time travel animation Pixar Cartoon Disney toy toys Pixar Pixar animation pixar animated animation comedy Disney Pixar ya boy clever computer animation Disney fantasy Pixar toys witty animation cgi rat

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(final['metadata'])
tfidf_df= pd.DataFrame(tfidf_matrix.toarray(),index=final.index.tolist())

In [53]:
print(tfidf_df.shape)

(13512, 21295)


In [54]:
tfidf_df

,0,1,2,3,4,5,6,7,8,9,...,21285,21286,21287,21288,21289,21290,21291,21292,21293,21294
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
tfidf_df.loc(0)

In [56]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 200)
latent_matrix = svd.fit_transform(tfidf_df)
# explained = svd.explained_varience_ratio.cunsum()

In [57]:
n=200
latent_matrix_l_df = pd.DataFrame(latent_matrix[:,0:n],index=final.title.tolist())
latent_matrix.shape

(13512, 200)

In [58]:
latent_matrix

array([[ 4.86551571e-02,  5.45150074e-02,  2.73885722e-02, ...,
        -1.76173777e-02, -2.12158809e-02, -2.44521746e-02],
       [ 2.42914060e-02,  1.09861969e-02,  4.47227687e-02, ...,
        -5.79036959e-03, -4.04206271e-03, -1.51421231e-02],
       [ 6.10138801e-02,  6.57547493e-02, -6.02443225e-04, ...,
        -5.27462253e-03, -6.58785915e-03,  4.21624741e-03],
       ...,
       [ 4.29819080e-01, -2.87452339e-01,  4.00509987e-01, ...,
        -2.31573308e-03, -1.23724881e-03, -4.17755471e-04],
       [ 7.66240107e-02,  1.00308724e-02,  1.22924198e-01, ...,
         1.57081952e-02,  8.47801446e-03,  2.53304712e-03],
       [ 4.04459204e-01, -2.81481140e-01,  6.24525033e-01, ...,
        -3.08445747e-03, -6.60260934e-04, -2.78113722e-03]])

In [59]:
ratings_f.head()

,userId,movieId,rating
0,1.0,2.0,3.5
1,1.0,29.0,3.5
2,1.0,32.0,3.5
3,1.0,47.0,3.5
4,1.0,50.0,3.5


In [60]:
ratings_f1=pd.merge(movies[['movieId']],ratings_f,on="movieId",how="right")
ratings_f2 = ratings_f1.pivot(index = 'movieId', columns = 'userId', values ='rating').fillna(0)
ratings_f2.head(3)

userId,1.0,2.0,3.0,5.0,7.0,8.0,11.0,13.0,14.0,16.0,...,5337.0,5338.0,5340.0,5341.0,5342.0,5346.0,5347.0,5348.0,5350.0,5352.0
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,4.0,4.5,4.0,4.5,3.0,...,3.5,5.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,2.5
2,3.5,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,...,2.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,1.5,2.0
3,0.0,4.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0


In [61]:
ratings_f2.shape

(13512, 3148)

In [62]:
len(ratings_f.movieId.unique())

13512

In [63]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 200)
latent_matrix_2 = svd.fit_transform(ratings_f2)
latent_matrix_2_df=pd.DataFrame(latent_matrix_2,index=final.title.tolist())

In [64]:
latent_matrix_2_df.shape

(13512, 200)

In [66]:
latent_matrix_l_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
Toy Story (1995),0.048655,0.054515,0.027389,-0.006518,0.067833,0.083618,-0.021187,0.130454,-0.009058,-0.047396,...,0.000392,0.012751,0.017477,-0.034827,0.025030,0.037758,-0.067937,-0.017617,-0.021216,-0.024452
Jumanji (1995),0.024291,0.010986,0.044723,0.002007,0.037822,0.072494,-0.003981,0.089086,-0.011436,-0.027433,...,-0.010023,-0.013637,0.008696,0.008723,-0.012727,0.017622,-0.020268,-0.005790,-0.004042,-0.015142
Grumpier Old Men (1995),0.061014,0.065755,-0.000602,0.009389,0.005679,0.022065,0.024684,-0.005944,-0.001466,-0.001171,...,0.001112,-0.004632,0.014514,0.023171,-0.004227,-0.000127,0.043820,-0.005275,-0.006588,0.004216
Waiting to Exhale (1995),0.169644,0.044606,-0.025022,0.035987,-0.006459,0.068136,0.088714,-0.035772,-0.005503,-0.011460,...,0.028688,-0.017077,0.042430,-0.071955,-0.085038,-0.028625,-0.036760,-0.039832,0.055551,0.033037
Father of the Bride Part II (1995),0.070181,0.082221,0.009964,-0.000787,0.016626,0.011374,-0.015768,0.025338,-0.001521,-0.002634,...,0.005083,0.017709,0.026829,0.029816,-0.038140,0.009862,0.005871,-0.014903,-0.017828,0.020515


In [67]:
latent_matrix_2_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
Toy Story (1995),114.007390,-1.573990,26.136520,15.953648,5.843151,35.990086,-3.399555,7.665011,15.240833,0.025738,...,-7.375534,-1.689081,-1.283045,0.294769,-1.951993,1.220536,0.170378,-3.099939,2.295018,-4.576868
Jumanji (1995),53.023975,-0.054846,31.700414,-6.739358,-10.451463,5.997618,-10.925832,10.561201,-1.872638,3.706518,...,0.553630,-1.470510,-1.845997,1.746891,-4.926869,-1.821506,2.093282,-1.237904,3.895177,-3.147956
Grumpier Old Men (1995),23.355985,-8.717053,16.705118,-8.190293,-9.342945,0.178811,-0.305697,-0.404921,2.195800,3.993192,...,0.071233,-1.118090,-0.263863,-0.086550,-1.332452,2.105816,-0.916604,1.751020,-0.306110,-0.317533
Waiting to Exhale (1995),5.476817,-5.402270,4.627437,-1.751648,-4.668420,-1.349797,1.742444,1.029241,1.315193,-1.755668,...,0.917847,0.550271,1.363507,0.372641,-0.312493,0.010547,-1.038395,0.042906,-1.120293,0.449491
Father of the Bride Part II (1995),20.414781,-8.791025,21.007581,-9.035887,-12.250778,1.195477,-0.447868,0.468119,2.131197,1.604280,...,-0.963992,0.005023,0.702520,1.091172,1.815177,-0.065184,1.558161,-2.090900,-0.156358,1.234216


In [75]:
from sklearn.metrics.pairwise import cosine_similarity
a_1 = np.array(latent_matrix_l_df.loc['Toy Story (1995)']).reshape(1,-1)
a_2 = np.array(latent_matrix_2_df.loc['Toy Story (1995)']).reshape(1,-1)

In [76]:
score_1=cosine_similarity(latent_matrix_l_df,a_1).reshape(-1)
score_2=cosine_similarity(latent_matrix_2_df,a_2).reshape(-1)

In [77]:
hybrid = ((score_1+score_2)/2.0)

In [78]:
dictDf = {'content':score_1,'colloborative':score_2,'hybrid':hybrid}
similar = pd.DataFrame(dictDf, index = latent_matrix_l_df.index)

In [80]:
similar.sort_values('hybrid',ascending =False , inplace = True)


In [81]:
similar[1:].head(11)

,content,colloborative,hybrid
Toy Story 2 (1999),0.966038,0.751993,0.859015
"Bug's Life, A (1998)",0.909306,0.669680,0.789493
"Monsters, Inc. (2001)",0.890918,0.626687,0.758803
Finding Nemo (2003),0.882317,0.605464,0.743890
Ice Age (2002),0.881194,0.474145,0.677669
"Incredibles, The (2004)",0.796920,0.557879,0.677400
Ratatouille (2007),0.898907,0.401276,0.650091
Antz (1998),0.751994,0.541300,0.646647
Toy Story 3 (2010),0.864853,0.379689,0.622271
Shrek (2001),0.574865,0.641949,0.608407


In [83]:
similar.sort_values('content',ascending =False , inplace = True)
similar[1:].head(11)

,content,colloborative,hybrid
Toy Story 2 (1999),0.966038,0.751993,0.859015
"Bug's Life, A (1998)",0.909306,0.669680,0.789493
Ratatouille (2007),0.898907,0.401276,0.650091
"Monsters, Inc. (2001)",0.890918,0.626687,0.758803
Finding Nemo (2003),0.882317,0.605464,0.743890
Ice Age (2002),0.881194,0.474145,0.677669
Toy Story 3 (2010),0.864853,0.379689,0.622271
Monsters University (2013),0.820516,0.169262,0.494889
Tin Toy (1988),0.799957,0.053521,0.426739
Red's Dream (1987),0.797502,0.053521,0.425512


In [84]:
similar.sort_values('colloborative',ascending =False , inplace = True)
similar[1:].head(11)

,content,colloborative,hybrid
Toy Story 2 (1999),0.966038,0.751993,0.859015
Forrest Gump (1994),0.275978,0.699971,0.487975
Aladdin (1992),0.389957,0.697274,0.543616
Jurassic Park (1993),0.059816,0.696833,0.378324
Back to the Future (1985),0.150203,0.695755,0.422979
"Lion King, The (1994)",0.430952,0.690957,0.560954
Independence Day (a.k.a. ID4) (1996),0.005165,0.689500,0.347332
Star Wars: Episode IV - A New Hope (1977),0.028591,0.687976,0.358283
Star Wars: Episode VI - Return of the Jedi (1983),0.016794,0.673369,0.345082
Mission: Impossible (1996),0.239535,0.670785,0.455160
